In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

% matplotlib inline

In [2]:
sales_df = pd.read_csv('./data/input/sales_train.csv')
tst_df = pd.read_csv('./data/input/test.csv')

**时间范围**

训练集的日期是从2013.1~2015.10总共34个月，是店铺每天卖出各个商品的数量

测试集只有2015.11这个月的数据，需要预测一整个月整体的商品出售数量

**测试集字段**

测试集只有`shop_id, item_id`这两个字段（`ID`没有意义，先忽略）

In [3]:
print('shape:', sales_df.shape)
sales_df.head()

shape: (2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
# 将sales数据按店铺-商品-月聚合
trn_df = sales_df.groupby(['shop_id', 'item_id', 'date_block_num']).sum().reset_index()
print('shape:', trn_df.shape)
trn_df.head()

shape: (1609124, 5)


,shop_id,item_id,date_block_num,item_price,item_cnt_day
0,0,30,1,2385.0,31.0
1,0,31,1,3038.0,11.0
2,0,32,0,884.0,6.0
3,0,32,1,1547.0,10.0
4,0,33,0,1041.0,3.0


In [5]:
print('shape:', tst_df.shape)
tst_df.head()

shape: (214200, 3)


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


**店铺数量**

可以看到训练集中每个月店铺数量都是在变化的，测试集中只有42个店铺，有很多店铺可能已经关了

但如果能提取商铺的一些属性作为特征，那么即便关店也是有他的价值

**商品数量**

可以看到训练集中每个月商品数量都在变化，趋势基本是下降的，最开始8000+，后面只有5000+

然后看到测试集数据只有5100个商品了

**记录数**

训练集中每个月的数据大约在30k~60k的样子，而测试集一个月的数据都有200k+，远远大于训练集（原因下面分析）

In [6]:
gb = trn_df.groupby('date_block_num')
pd.DataFrame({
    'size': gb.size(),
    'shop_num': gb['shop_id'].nunique(),
    'item_num': gb['item_id'].nunique(),
})

,item_num,shop_num,size
date_block_num,,,
0,8115,45,63224
1,8168,46,59935
2,8302,46,63977
3,8145,46,54638
4,8339,45,53296
5,8478,46,56196
6,8405,46,58035
7,8031,45,58022
8,7876,45,51575


In [7]:
# 测试集中店铺数量
tst_df['shop_id'].nunique()

42

In [8]:
# 测试集中商品数量
tst_df['item_id'].nunique()

5100

In [9]:
# 测试集记录数
tst_df['ID'].count()

214200

**测试集每月记录数远大于训练集每月记录数原因分析**

测试集中，每个`shop_id`对应的记录都是5100条

同时，可以看到售卖记录中并没有出售0个的记录

说明测试集中需要预测的是，每个店铺对应每个商品的出售数量（即便这个商店可能不卖某些商品）

In [10]:
tst_df.groupby(['shop_id']).size()

shop_id
2     5100
3     5100
4     5100
5     5100
6     5100
7     5100
10    5100
12    5100
14    5100
15    5100
16    5100
18    5100
19    5100
21    5100
22    5100
24    5100
25    5100
26    5100
28    5100
31    5100
34    5100
35    5100
36    5100
37    5100
38    5100
39    5100
41    5100
42    5100
44    5100
45    5100
46    5100
47    5100
48    5100
49    5100
50    5100
52    5100
53    5100
55    5100
56    5100
57    5100
58    5100
59    5100
dtype: int64

In [11]:
# 售卖记录是否有出售0个商品的记录
(sales_df['item_cnt_day'] == 0).sum()

0

# 验证集策略

测试集是2015.11这个月的数据，需要预测所有店铺所有商品的销量

那么验证集可以为2015.10这个月，所有店铺所有商品的数据，如果没有记录就补0

**不好的验证集策略**

刚开始使用了随机分割，验证集sorce=5.5左右，而LB的socre=1.5左右，差距很大，难以minic测试集

即便使用2015.10全月的数据作为验证集仍然差距很大，难以minic测试集